Feature Engineering編での特徴量の処理を完了していることを前提としています

## メインモデル

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import datetime
import gc
import os

train_df.head()

In [ ]:
target = train_df['meter_reading']
features = train_df.drop('meter_reading', axis=1)
del train_df
gc.collect()

In [ ]:
# カテゴリを表す特徴量を指定（以下の特徴は数値で表現してありますが、それらは数値の大小は関係なく、単に種類を表しています）
categorical_features = ["building_id", "site_id", "meter", 'building_meter_category', 'primary_use']

In [ ]:
# objective: タスク種別の指定（回帰、2値分類、多クラス分類など）
# num_leaves: 用いる決定木の個数。小さすぎると学習が足りず、大きすぎると過学習に繋がる恐れがあります
# learning_rate: 学習率
# metric: 誤差定義

params = {
    "objective": "regression",
    "boosting": "gbdt",
    "num_leaves": 1300,
    "learning_rate": 0.05,
    "feature_fraction": 0.8,
    "reg_lambda": 2,
    "metric": "rmse",
}

In [ ]:
# k-Foldでデータセットを何分割するか指定します(今回は3セット)
kf = KFold(n_splits=3)
# データセットの分割数分モデルができるので、そのモデルを格納する入れ物を作成
models = []

# 指定した分割数で特徴量を分割し、分割数分ループを回しながら学習します
# kf.split()で学習用のデータと検証用のデータのindexが返されます
for train_index, test_index in kf.split(features):
    # index指定でデータを分けます（目的変数と学習用特徴量それぞれ）
    train_features = features.loc[train_index]
    train_target = target.loc[train_index]
    
    test_features = features.loc[test_index]
    test_target = target.loc[test_index]
    
    # lightGBMが学習できる形にします
    d_training = lgb.Dataset(train_features, label=train_target, categorical_feature=categorical_features, free_raw_data=False)
    d_test = lgb.Dataset(test_features, label=test_target, categorical_feature=categorical_features, free_raw_data=False)
    
    # lgb.train()で学習用・検証用データを渡し、学習させます
    # num_boost_round: ブースティングの反復回数
    # verbose_eval: どのくらいの頻度で出力を示すかの指定
    # early_stopping_rounds: 検証用のスコアが改善されなくなったら学習を停止させるため（過学習を抑える）
    model = lgb.train(params, train_set=d_training, num_boost_round=1000, valid_sets=[d_training, d_test], verbose_eval=25, early_stopping_rounds=50)

    # 学習したモデルを格納します
    models.append(model)
    del train_features, train_target, test_features, test_target, d_training, d_test
    gc.collect()

In [ ]:
for model in models:
    # lightGBMのplot_importanceメソッドにmodel学習したmodelを渡します
    lgb.plot_importance(model)
    plt.show()

In [ ]:
test_df = pd.read_csv("../input/ashrae-energy-prediction/test.csv")
row_ids = test_df["row_id"]
test_df.drop("row_id", axis=1, inplace=True)
test_df = reduce_mem_usage(test_df)

test_df = test_df.merge(building_meta_df, left_on="building_id", right_on="building_id", how="left")
del building_meta_df
gc.collect()

weather_df = pd.read_csv("../input/ashrae-energy-prediction/weather_test.csv")
weather_df = fill_weather_dataset(weather_df)
weather_df = get_meteorological_features(weather_df)
weather_df = reduce_mem_usage(weather_df)

test_df = test_df.merge(weather_df, how="left", on=["timestamp", "site_id"])
del weather_df
gc.collect()

In [ ]:
# 分割したモデルでそれぞれ予測した値を格納する入れ物を準備する
results = []

# モデルはリストとして格納してあり、ループを回すことでそれぞれのモデルを取り出す
for model in models:
    if results == []:
        # 予測値を計算
        # 各モデルで予測した平均値を最終的な予測値とするため、モデル数で割った値を保存する
        results = model.predict(test_df, num_iteration=model.best_iteration)/ len(models)
    else:
        results += model.predict(test_df, num_iteration=model.best_iteration)/len(models)
    del model
    gc.collect()

del test_df, models
gc.collect()

In [ ]:
# 特徴量エンジニアリングで"meter_reading"は対数変換していたため、元に戻す
results = np.expm1(results)

# データフレーム型に変換し、提出フォーマットを整える
results_df = pd.DataFrame({"row_id": row_ids, "meter_reading": np.clip(results, 0, a_max=None)})
del row_ids, results
gc.collect()

# csvファイルとして保存する
results_df.to_csv("submission_noleak.csv", index=False)

## その他の手法
### メーターごとのモデル作成方法

In [ ]:
# meter=0のモデルの学習

# train_df内のmeter==0に該当するデータのみ抽出してfeatures_0という変数で格納する
features_0 = train_df[train_df.meter==0]
target_0 = train_df['meter_reading']
features_0 = train_df.drop(['meter_reading'], axis=1)

In [ ]:
categorical_features = ["building_id", "site_id", "meter", 'building_meter_category', 'primary_use']

params = {
    "objective": "regression",
    "boosting": "gbdt",
    "num_leaves": 1000,
    "learning_rate": 0.05,
    "feature_fraction": 0.8,
    "reg_lambda": 2,
    "metric": "rmse",
}

kf = KFold(n_splits=3)
models = []

for train_index, test_index in kf.split(features_0):
    train_features = features_0.loc[train_index]
    train_target = target_0.loc[train_index]
    
    test_features = features_0.loc[test_index]
    test_target = target_0.loc[test_index]
    
    d_training = lgb.Dataset(train_features, label=train_target, categorical_feature=categorical_features, free_raw_data=False)
    d_test = lgb.Dataset(test_features, label=test_target, categorical_feature=categorical_features, free_raw_data=False)
    
    model = lgb.train(params, train_set=d_training, num_boost_round=1000, valid_sets=[d_training, d_test], verbose_eval=25, early_stopping_rounds=50)

    models.append(model)
    del train_features, train_target, test_features, test_target, d_training, d_test
    gc.collect()

In [ ]:
# 先にメモリを解放しておきます
del train_df, target_0, features_0
gc.collect()


In [ ]:
# テストデータのうち、meter=0のもののみ抽出する
test_0 = test_df[test_df.meter==0]

# 抽出したのち'row_id'を取り出す
row_ids_0 = test_0["row_id"]
test_0 = test_0.drop(['row_id'], axis=1)


# モデルはリストとして格納してあり、ループを回すことでそれぞれのモデルを取り出す
for model in models:
    if results_0 == []:
        results_0 = model.predict(test_0, num_iteration=model.best_iteration)/ len(models)
    else:
        results_0 += model.predict(test_0, num_iteration=model.best_iteration)/len(models)
    del model
    gc.collect()

del test_0, models
gc.collect()

results_0 = np.expm1(results_0)

# データフレーム型に変換し、提出フォーマットを整える
results0_df = pd.DataFrame({"row_id": row_ids_0, "meter_reading": np.clip(results_0, 0, a_max=None)})
del row_ids_0, results_0
gc.collect()